# COTS Detection Training code

In [1]:
# 安装所需的库
# Install the required libraries
!pip install -qU wandb
!pip install -qU bbox-utility

In [2]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
from joblib import Parallel, delayed
from IPython.display import display

sys.path.append('../input/tensorflow-great-barrier-reef')

In [3]:
# 登录 WANDB
# login WANDB
import wandb
wandb.login()

In [4]:
# 训练设置
# Training
FOLD      = 1  # 用来训练的FOLD  # which fold to train
DIM       = 3000  # 输入尺寸  # Input size
MODEL     = 'yolov5s6'  # 模型  # model
BATCH     = 4
EPOCHS    = 10
OPTMIZER  = 'Adam'

PROJECT   = 'great-barrier-reef-public' # w&b in yolov5
NAME      = f'{MODEL}-dim{DIM}-fold{FOLD}' # w&b for yolov5

REMOVE_NOBBOX = True  # 去除没有bbox的背景图片  # remove images with no bbox
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
IMAGE_DIR = '/kaggle/images' # directory to save images
LABEL_DIR = '/kaggle/labels' # directory to save labels

## 创建目录 Create Directories

In [5]:
!mkdir -p {IMAGE_DIR}
!mkdir -p {LABEL_DIR}

In [15]:
# Train Data
df = pd.read_csv(f'{ROOT_DIR}/train.csv')
df['old_image_path'] = f'{ROOT_DIR}/train_images/video_'+df.video_id.astype(str)+'/'+df.video_frame.astype(str)+'.jpg'
df['image_path']  = f'{IMAGE_DIR}/'+df.image_id+'.jpg'
df['label_path']  = f'{LABEL_DIR}/'+df.image_id+'.txt'
df['annotations'] = df['annotations'].progress_apply(eval)

In [16]:
print(df.old_image_path[1])
print(df.image_path[1])
print(df.label_path[1])

## 检查bbox数量 Number of BBoxes

In [17]:
df['num_bbox'] = df['annotations'].progress_apply(lambda x: len(x))
data = (df.num_bbox>0).value_counts(normalize=True)*100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")

In [18]:
if REMOVE_NOBBOX:
    df = df.query("num_bbox>0")

# 写入图片 Write Images
* 将图片保存到 `/kaggle/working`  copy the Images to Current Directory `/kaggle/working` 


In [19]:
def make_copy(row):
    shutil.copyfile(row.old_image_path, row.image_path)
    return

In [20]:
image_paths = df.old_image_path.tolist()
_ = Parallel(n_jobs=-1, backend='threading')(delayed(make_copy)(row) for _, row in tqdm(df.iterrows(), total=len(df)))

## 制作数据集 Make the dataset

In [21]:
# check https://github.com/awsaf49/bbox for source code of following utility functions
from bbox.utils import coco2yolo, coco2voc, voc2yolo
from bbox.utils import draw_bboxes, load_image
from bbox.utils import clip_bbox, str2annot, annot2str

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row

np.random.seed(32)
colors = [(np.random.randint(255), np.random.randint(255), np.random.randint(255))\
          for idx in range(1)]

In [22]:
df['bboxes'] = df.annotations.progress_apply(get_bbox)
df.head(2)

## 图片尺寸 Image-Size
* 尺寸为 `[1280, 720]` The size is `[1280, 720]`

In [23]:
df['width']  = 1280
df['height'] = 720
display(df.head(2))

# 创建标签 Create Labels
将标签保存为 **YOLO** 格式  
Export labels to **YOLO** format.

* 一行为一个目标 One row per object
* 每一行的格式 Each row is class `[x_center, y_center, width, height]` format.
* 数据归一化 Data normalized
* 类的数字从 0 开始 Class numbers are **zero-indexed**.

In [25]:
cnt = 0
all_bboxes = []
bboxes_info = []
for row_idx in tqdm(range(df.shape[0])):
    row = df.iloc[row_idx]
    image_height = row.height
    image_width  = row.width
    bboxes_coco  = np.array(row.bboxes).astype(np.float32).copy()
    num_bbox     = len(bboxes_coco)
    names        = ['cots']*num_bbox
    labels       = np.array([0]*num_bbox)[..., None].astype(str)
    ## Create Annotation(YOLO)
    with open(row.label_path, 'w') as f:
        if num_bbox<1:
            annot = ''
            f.write(annot)
            cnt+=1
            continue
        bboxes_voc  = coco2voc(bboxes_coco, image_height, image_width)
        bboxes_voc  = clip_bbox(bboxes_voc, image_height, image_width)
        bboxes_yolo = voc2yolo(bboxes_voc, image_height, image_width).astype(str)
        all_bboxes.extend(bboxes_yolo.astype(float))
        bboxes_info.extend([[row.image_id, row.video_id, row.sequence]]*len(bboxes_yolo))
        annots = np.concatenate([labels, bboxes_yolo], axis=1)
        string = annot2str(annots)
        f.write(string)
print('Missing:',cnt)

# 创建folds Create Folds


In [26]:
from sklearn.model_selection import GroupKFold
kf = GroupKFold(n_splits = 3)
df = df.reset_index(drop=True)
df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df, groups=df.video_id.tolist())):
    df.loc[val_idx, 'fold'] = fold
display(df.fold.value_counts())

# 数据集展示 Visualization

In [31]:
df2 = df[(df.num_bbox>0)].sample(100) # takes samples with bbox
y = 2; x = 2
plt.figure(figsize=(12.8*x, 7.2*y))
for idx in range(x*y):
    row = df2.iloc[idx]
    img           = load_image(row.image_path)
    image_height  = row.height
    image_width   = row.width
    with open(row.label_path) as f:
        annot = str2annot(f.read())
    bboxes_yolo = annot[...,1:]
    labels      = annot[..., 0].astype(int).tolist()
    names         = ['cots']*len(bboxes_yolo)
    plt.subplot(y, x, idx+1)
    plt.imshow(draw_bboxes(img = img,
                           bboxes = bboxes_yolo, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = 'yolo',
                           line_thickness = 2))
    plt.axis('OFF')
plt.tight_layout()
plt.show()

# 制作数据集 Make Dataset

In [32]:
train_files = []
val_files   = []
train_df = df.query("fold!=@FOLD")
valid_df = df.query("fold==@FOLD")
train_files += list(train_df.image_path.unique())
val_files += list(valid_df.image_path.unique())
len(train_files), len(val_files)

# 训练配置 Configuration

In [33]:
import yaml

cwd = '/kaggle/working/'

with open(os.path.join( cwd , 'train.txt'), 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path+'\n')
            
with open(os.path.join(cwd , 'val.txt'), 'w') as f:
    for path in valid_df.image_path.tolist():
        f.write(path+'\n')

data = dict(
    path  = '/kaggle/working',
    train =  os.path.join( cwd , 'train.txt') ,
    val   =  os.path.join( cwd , 'val.txt' ),
    nc    = 1,
    names = ['cots'],
    )

with open(os.path.join( cwd , 'gbr.yaml'), 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

f = open(os.path.join( cwd , 'gbr.yaml'), 'r')
print('\nyaml:')
print(f.read())

## 训练参数 Parameters

In [36]:
%%writefile /kaggle/working/hyp.yaml
lr0: 0.01  # 学习率 initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # 最终学习率 final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD动量 SGD momentum/Adam beta1
weight_decay: 0.0005  # 优化器权重衰减 optimizer weight decay 5e-4
warmup_epochs: 3.0  # 热启动 warmup epochs
warmup_momentum: 0.8  # 热启动初始动量 warmup initial momentum
warmup_bias_lr: 0.1  # 热启动初始偏差lr warmup initial bias lr
box: 0.05  # box损失增益 box loss gain
cls: 0.5  # cls损失增益 cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
hsv_h: 0.015  # 图像 HSV-Hue 增强 image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # 图像 HSV 饱和度增强 image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # 图像 HSV 值增强 image HSV-Value augmentation (fraction)
degrees: 0.0  # 图像旋转 image rotation (+/- deg)
translate: 0.10  # 平移 translate
scale: 0.5  # 缩放 scale
shear: 0.0  # 左右偏移 shear
perspective: 0.0  # 图像透视 image perspective
flipud: 0.5  # 上下翻转 Flip upside down(probability)
fliplr: 0.5  # 左右翻转 Flip left and right(probability)
mosaic: 0.5  # 马赛克 mosaic(probability)
mixup: 0.5   # mixup (probability)
copy_paste: 0.0  # 复制粘贴 segment copy-paste (probability)

In [35]:
%cd /kaggle/working
!rm -r /kaggle/working/yolov5
# !git clone https://github.com/ultralytics/yolov5
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5
%cd yolov5
%pip install -qr requirements.txt  # install

from yolov5 import utils
display = utils.notebook_init()  # check

# 训练 Training

In [ ]:
!python train.py --img {DIM}\
--batch {BATCH}\
--epochs {EPOCHS}\
--optimizer {OPTMIZER}\
--data /kaggle/working/gbr.yaml\
--hyp /kaggle/working/hyp.yaml\
--weights {MODEL}.pt\
--project {PROJECT} --name {NAME}\
--exist-ok

# 查看结果 Overview
<span style="color: #000508; font-family: Segoe UI; font-size: 1.5em; font-weight: 300;"><a href="https://wandb.ai/awsaf49/great-barrier-reef-public">点击在此网站查看训练结果 View the Complete Dashboard Here</a></span>


In [ ]:
OUTPUT_DIR = '{}/{}'.format(PROJECT, NAME)
!ls {OUTPUT_DIR}

In [ ]:
!rm -r {IMAGE_DIR}
!rm -r {LABEL_DIR}